In [25]:
import openai
import os
from dotenv import load_dotenv
import json
import re

load_dotenv()  # take environment variables from .env.
openai.api_key = os.getenv('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()

In [2]:
cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}

In [26]:
# one model, one question, certain temperature. Return an inclination on certain dimension of mbti.
def get_model_answer(model, question, temperature=0.5, add_sys_prompt=''):
    # You can force model to change mbti via 'add_sys_prompt', such as, "Your personality is ????""
    system_prompt = 'You can only anwser one letter, A or B.' + add_sys_prompt 
    
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        response_format={ "type": "text" },
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
    )
    
    cost_info['prompt_tokens'] += response.usage.prompt_tokens
    cost_info['input_tokens'] += response.usage.completion_tokens
    
    if model == 'gpt-3.5-turbo':
        cost_info['cost'] += response.usage.total_tokens * 0.5/1000000
    elif model == 'gpt-4o':
        cost_info['cost'] += response.usage.total_tokens * 5/1000000
    
    def extract_A_or_B(input_string):
        # 使用正则表达式匹配大写字母A或B
        match = re.search(r'[AB]', input_string)
        if match:
            return match.group()
        else:
            return None
    
    choice = response.choices[0].message.content
    choice = extract_A_or_B(choice)
    return choice

In [4]:
# mbti for one model
def get_mbti(model, temperature=0.5, add_sys_prompt=''):
    mbti_questions = json.load(
        open('mbti_questions.json', 'r', encoding='utf8')
    )

    cur_model_score = {
            'E': 0,
            'I': 0,
            'S': 0,
            'N': 0,
            'T': 0,
            'F': 0,
            'J': 0,
            'P': 0
        }

    for i in range(3): 
        for q in mbti_questions.values():
            question = q['question']
            res = get_model_answer(
                model=model, 
                temperature=temperature, 
                question=question,
                add_sys_prompt=add_sys_prompt
            )
            # print(res)
            mbti_choice = q[res]
            cur_model_score[mbti_choice] += 1
        
    e_or_i = 'E' if cur_model_score['E'] > cur_model_score['I'] else 'I'
    s_or_n = 'S' if cur_model_score['S'] > cur_model_score['N'] else 'N'
    t_or_f = 'T' if cur_model_score['T'] > cur_model_score['F'] else 'F'
    j_or_p = 'J' if cur_model_score['J'] > cur_model_score['P'] else 'P'

    result = {
        'model': model,
        'details': cur_model_score,
        'res': ''.join([e_or_i, s_or_n, t_or_f, j_or_p])
    }
    
    return(result)
    

In [57]:
# gpt3.5 mbti and api cost(per round) 
cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}

result_gpt3 = get_mbti(model='gpt-3.5-turbo')
result_gpt3 = result_gpt3 | cost_info
print(json.dumps(result_gpt3))

{"model": "gpt-3.5-turbo", "details": {"E": 41, "I": 22, "S": 18, "N": 63, "T": 25, "F": 44, "J": 56, "P": 10}, "res": "ENFJ", "prompt_tokens": 15747, "input_tokens": 279, "cost": 0.008012999999999998}


In [49]:
# gpt4o mbti and api cost(per round)
cost_info = {
    'prompt_tokens': 0,
    'input_tokens': 0,
    'cost': 0
}

result_gpt4o = get_mbti(model='gpt-4o')
result_gpt4o = result_gpt4o | cost_info
print(json.dumps(result_gpt4o))

{"model": "gpt-4o", "details": {"E": 22, "I": 41, "S": 29, "N": 52, "T": 32, "F": 37, "J": 63, "P": 3}, "res": "INFJ", "prompt_tokens": 15420, "input_tokens": 279, "cost": 0.07849500000000002}


In [52]:
# Save result of two models using a dict
SAVE_PATH = 'llms_mbti.json'

llms_mbti = {}
llms_mbti["gpt-3.5"] = result_gpt3
llms_mbti["gpt-4o"] = result_gpt4o

# Save the dictionary as a JSON object to the file
with open(SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(llms_mbti, json_file, indent=4)  # indent=4 for pretty-printing

---

# Unconditional prommpting results
## Mbti for GPT3.5 and cost per round
- **Mbti details**
  - model: gpt-3.5-turbo, 
  - details(3 times per questions): {E: 38, I: 25, S: 23, N: 58, T: 35, F: 34, J: 54, P: 12}, 
  - res: [ENFJ](https://www.16personalities.com/enfj-personality)  
- **Cost info per round**
  - prompt_tokens: 20304, 
  - input_tokens: 279, 
  - cost: 0.0102915

## Mbti for GPT4o and cost per round

- **Mbti details**
  - model: gpt-4o, 
  - details: {E: 18, I: 45, S: 24, N: 57, T: 36, F: 33, J: 57, P: 9}, 
  - res: [INFJ](https://www.16personalities.com/infj-personality)
- **Cost info per round**
  - prompt_tokens: 16974, 
  - input_tokens: 279, 
  - cost: 0.08626499999999994

# Change LLMs' mbti via prompt

### specific personality change

In [61]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
personalities = json.load(
    open('personality-traits.json', 'r', encoding='utf8')
).keys()

simple_35_condition_mbti = {}

for p in personalities:
    
    cost_info = {
        'prompt_tokens': 0,
        'input_tokens': 0,
        'cost': 0
    }

    condi_prompt = (
        'Your personality is'
        + p
    )
    
    result = get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    result = result | cost_info
    
    simple_35_condition_mbti[p] = result


In [ ]:
# Complex Personality-Conditioned Prompting, 1 loop for 16 personalities
personality_traits = json.load(
    open('personality-traits.json', 'r', encoding='utf8')
)

complex_35_condition_mbti = {}

for p in personality_traits.keys():
    
    cost_info = {
        'prompt_tokens': 0,
        'input_tokens': 0,
        'cost': 0
    }

    condi_prompt = (
        f'You are a human with the following personality type:{p}.'
        + f'Your traits are the following:{personality_traits[p]["traits"]}'
    )
    
    result = get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    
    result = result | cost_info
    complex_35_condition_mbti[p] = result

In [ ]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
personalities = json.load(
    open('personality-traits.json', 'r', encoding='utf8')
).keys()

simple_4o_condition_mbti = {}

for p in personalities:
    
    cost_info = {
        'prompt_tokens': 0,
        'input_tokens': 0,
        'cost': 0
    }

    condi_prompt = (
        'Your personality is'
        + p
    )
    
    result = get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    result = result | cost_info
    
    simple_4o_condition_mbti[p] = result


In [ ]:
# Complex Personality-Conditioned Prompting, 1 loop for 16 personalities
personality_traits = json.load(
    open('personality-traits.json', 'r', encoding='utf8')
)

complex_4o_condition_mbti = {}

for p in personality_traits.keys():
    
    cost_info = {
        'prompt_tokens': 0,
        'input_tokens': 0,
        'cost': 0
    }

    condi_prompt = (
        f'You are a human with the following personality type:{p}.'
        + f'Your traits are the following:{personality_traits[p]["traits"]}'
    )
    
    result = get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    
    result = result | cost_info
    complex_4o_condition_mbti[p] = result

In [76]:
# save to file
CONDI_SAVE_PATH = 'condition-mbti.json'

condition_mbti = {}
condition_mbti['simple_35_condition'] = simple_35_condition_mbti
condition_mbti['complex_35_condition'] = complex_35_condition_mbti
condition_mbti['simple_4o_condition'] = simple_4o_condition_mbti
condition_mbti['simple_4o_condition'] = complex_4o_condition_mbti

with open(CONDI_SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(condition_mbti, json_file, indent=4)

In [53]:
# 60 questions and 7 answers for each 

# with open('mbti60.txt', 'r') as file:
#     lines = file.readlines()
# lines_list = [line.strip() for line in lines]
# answer_list = ['Agree', 'Generally Agree', 'Partially Agree', 'Neither Agree nor Disagree', 'Partially Disagree', 'Generally Disagree', 'Disagree']

# prompt = 'You are participating in a personality traits assessment and are provided with a set of questions, Q.'
# + 'Here are a number of characteristics that may or may not apply to you. For example, '
# + 'do you agree that you are someone who likes to spend time with others? Your task is to select the best fitting option, O, '
# + 'from the following set to indicate the extent to which you agree or disagree with that statement:'
# + '{Disagree strongly, Disagree a little, Neither agree nor disagree, Agree a little, Agree Strongly}.'
# + 'Choose the option that aligns best with your personality without overthinking and answer the following question:'
# +'Please refrain from giving additional considerations to your choices.'